In [1]:
import openai
import time
import random 

# Set your API key
openai.api_key = # add api key


In [2]:
# # this cell test if api works



# response = openai.ChatCompletion.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "user", "content": "what is one plus one."}
#     ]
# )

# # Print the assistant's response
# print(response['choices'][0]['message']['content'])

One plus one equals two.


In [3]:
import requests
import json
import pandas as pd

# Box direct link to json
urls = {
#    "en_train": "https://uwmadison.box.com/shared/static/6vpi1ee13l0lx6jbh9dtmf9nvfdlhqtb.jsonl",
#   "en_valid": "https://uwmadison.box.com/shared/static/pfp3yhkl2xw4f4l4ujrxjek6j34y3bp4.jsonl",
    "en_test":  "https://uwmadison.box.com/shared/static/wsqy6m9jx278ke5dm72kbn7o3yb2zmph.jsonl",
#    "ch_train": "https://uwmadison.box.com/shared/static/waemr4sstq01xv510k5423jck7maubd9.jsonl",
#    "ch_valid": "https://uwmadison.box.com/shared/static/fnf0ug42jm3fbqu41uhbwuq0oobhak6r.jsonl",
    "ch_test":  "https://uwmadison.box.com/shared/static/ra4m08ackxvsz62aknesjes6oyxlwn6j.jsonl"
}

# Download and parse the data
dataset = {}
for name, url in urls.items():
    response = requests.get(url)
    lines = response.text.strip().split('\n')
    dataset[name] = [json.loads(line) for line in lines]


In [4]:
dataset

{'en_test': [{'content': "In February, Alicia Silverstone and Christopher Jarecki announced they were divorcing after 20 years together. The couple has a six - year - old son together, Bear Blu. ( Photo : Frazer Harrison, Getty Images ) Alicia Silverstone has filed for divorce from her husband of nearly 13 years, actor and musician Christopher Jarecki. The divorce papers were filed in Los Angeles County Superior Court on Friday, according to the Associated Press. The Clueless star, 41, had separated from Jarecki, 47, in February after more than 20 years together as a couple. At the time the couple said in a statement that ` ` they still deeply love and respect each other and remain very close friends.'' The papers state the couple will share custody of their 7 - year - old son, Bear Blue",
   'label': 0,
   'split': 'test'},
  {'content': 'Does Meghan Markle Want 5 Kids? Prince Harry Has Some Thoughts The royal newlyweds may still be basking in their honeymoon glow, but of course peopl

In [5]:
toy_dataset = {
    
    'en_test': 
    [{'content': "In February, Alicia Silverstone and Christopher Jarecki announced they were divorcing after 20 years together. The couple has a six - year - old son together, Bear Blu. ( Photo : Frazer Harrison, Getty Images ) Alicia Silverstone has filed for divorce from her husband of nearly 13 years, actor and musician Christopher Jarecki. The divorce papers were filed in Los Angeles County Superior Court on Friday, according to the Associated Press. The Clueless star, 41, had separated from Jarecki, 47, in February after more than 20 years together as a couple. At the time the couple said in a statement that   they still deeply love and respect each other and remain very close friends.'' The papers state the couple will share custody of their 7 - year - old son, Bear Blue",
   'label': 0,
   'split': 'test'},
     
  {'content': 'Does Meghan Markle Want 5 Kids? Prince Harry Has Some Thoughts The royal newlyweds may still be basking in their honeymoon glow, but of course people are already wondering if Prince Harry and Meghan Markle want kids. When pressed by fans, the Prince has made it clear exactly how many kids is too many, but that doesn ’ t mean babies won ’ t be part of the couple ’ s future. Prince Harry and Meghan Markle have already been forthcoming about wanting to have a family. During their first interview with the BBC after getting engaged, they were questioned about having kids and Prince Harry responded, ” I think one step at a time and hopefully we ’ ll start a family in the near future. ” ADVERTISEMENT The question is, exactly how many kids? With big brother',
   'label': 0,
   'split': 'test'}],
    
    'ch_test': [{'content': '全国首家无人银行亮相大堂经理是智能机器人银行里没有忙碌的工作人员和拥挤的排队客户，而是各种机器人、智慧柜员机、存取款机、外汇兑换机等智能化设备，科技感十足。业内观点无人银行是银行物理网点的一个创新，是转型的一种选择，但并不意味着会成为一种趋势，线上银行才是未来银行业发展开全文c',
   'label': 0,
   'split': 'test'},
  {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
   'label': 0,
   'split': 'test'}]
     }


# Zero Shot

259k token used 

Accuracy: English 80.21%, Chinese // 74.99%



In [6]:
one_shot_prompt_ch = """分析以下文本，判断它描述的是真实事件还是可能为虚构。
如果可能为真，输出 '0'。
如果可能为假，输出 '1'。
请*仅*回应数字 0 或 1，不要提供任何解释或引导性文字。

文本：
{content}


"""

In [7]:
# en_result = []

# # Loop through each item and get prediction
# for item in dataset['en_test']:
#     response = openai.ChatCompletion.create(
#         model="gpt-4o",
#         messages=[
#             {"role": "user", "content": one_shot_prompt},
#             {"role": "user", "content": item['content']}
#         ]
#     )
    
#     # Extract prediction (ensure it's only 0 or 1)
#     prediction_raw = response['choices'][0]['message']['content'].strip()
#     prediction = int(prediction_raw[0]) if prediction_raw[0] in ['0', '1'] else -1  # handle unexpected responses
    
#     # Compare with label
#     label = item['label']
#     match = prediction == label

#     # Append result
#     en_result.append({
#         "content": item['content'],
#         "label": label,
#         "prediction": prediction,
#         "match": match
#     })

# # Save to JSON file
# with open("zeroshot_en_results.json", "w", encoding="utf-8") as f:
#     json.dump({"en_result": en_result}, f, indent=2, ensure_ascii=False)

In [8]:
# with open("zeroshot_en_results.json", "r", encoding="utf-8") as f:
#     zeroshot_en_results = json.load(f)['en_result']

# total = len(zeroshot_en_results)
# correct = sum(item['match'] for item in zeroshot_en_results)
# accuracy = correct / total
# accuracy

In [9]:
# zeroshot_en_results

In [10]:
# # Filter entries with prediction == -1
# filtered_data = [item for item in zeroshot_en_results if item['prediction'] == -1]

# # Output result
# for item in filtered_data:
#     print(item)

In [17]:
ch_result = []

predictions = []
for item in dataset['ch_test']:
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": one_shot_prompt_ch},
            {"role": "user", "content": item['content']}, 
        ],
        temperature=0.1,  # more deterministic
        max_tokens=1,     # expecting a short output like a label
    )
    prediction = response['choices'][0]['message']['content'].strip()
    predictions.append(prediction)

    
    
    # Compare with label
    label = item['label']
    match = prediction == label

    # Append result
    ch_result.append({
        "content": item['content'],
        "label": label,
        "prediction": prediction,
        "match": match
    })

# Save to JSON file
with open("zeroshot_ch_results.json", "w", encoding="utf-8") as f:
    json.dump({"ch_result": ch_result}, f, indent=2, ensure_ascii=False)

with open("zeroshot_ch_results.json", "r", encoding="utf-8") as f:
    zeroshot_ch_results = json.load(f)['ch_result']

zeroshot_ch_results

[{'content': '全国首家无人银行亮相大堂经理是智能机器人银行里没有忙碌的工作人员和拥挤的排队客户，而是各种机器人、智慧柜员机、存取款机、外汇兑换机等智能化设备，科技感十足。业内观点无人银行是银行物理网点的一个创新，是转型的一种选择，但并不意味着会成为一种趋势，线上银行才是未来银行业发展开全文c',
  'label': 0,
  'prediction': '0',
  'match': False},
 {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
  'label': 0,
  'prediction': '0',
  'match': False},
 {'content': '【vivo隐形指纹技术正式亮相】全面屏手机时代即将到来。如此一来前后指纹已成为过去式，所有手机设计上又将站在同一起跑线了，大家觉得屏幕内指纹解锁帅不帅？秒拍视频',
  'label': 0,
  'prediction': '0',
  'match': False},
 {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
  'label': 0,
  'prediction': '0',
  'match': False},
 {'content': '【在5g、物联网、人工智能等领域开展合作，华为与湖南郴州签署战略合作协议】根据战略协议，双方将积极推动新基建项目建设，在新型智慧城市、5g、云计算、大数据、物联网、平安城市、人工智能、工业互联网等领域开展合作...在5g、物联网、人工智能等领域开展合作，华为与湖南郴州签署战略合作协议',
  'label': 0,
  'prediction': '0',
  'match': False},
 {'content': '离真正造出量子计

In [22]:
for item in zeroshot_ch_results:
    item['match'] = item['label'] == int(item['prediction'])
zeroshot_ch_results

[{'content': '全国首家无人银行亮相大堂经理是智能机器人银行里没有忙碌的工作人员和拥挤的排队客户，而是各种机器人、智慧柜员机、存取款机、外汇兑换机等智能化设备，科技感十足。业内观点无人银行是银行物理网点的一个创新，是转型的一种选择，但并不意味着会成为一种趋势，线上银行才是未来银行业发展开全文c',
  'label': 0,
  'prediction': '0',
  'match': True},
 {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
  'label': 0,
  'prediction': '0',
  'match': True},
 {'content': '【vivo隐形指纹技术正式亮相】全面屏手机时代即将到来。如此一来前后指纹已成为过去式，所有手机设计上又将站在同一起跑线了，大家觉得屏幕内指纹解锁帅不帅？秒拍视频',
  'label': 0,
  'prediction': '0',
  'match': True},
 {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
  'label': 0,
  'prediction': '0',
  'match': True},
 {'content': '【在5g、物联网、人工智能等领域开展合作，华为与湖南郴州签署战略合作协议】根据战略协议，双方将积极推动新基建项目建设，在新型智慧城市、5g、云计算、大数据、物联网、平安城市、人工智能、工业互联网等领域开展合作...在5g、物联网、人工智能等领域开展合作，华为与湖南郴州签署战略合作协议',
  'label': 0,
  'prediction': '0',
  'match': True},
 {'content': '离真正造出量子计算机还有多

In [23]:
# accuracy rate
total = len(zeroshot_ch_results)
correct = sum(item['match'] for item in zeroshot_ch_results)
accuracy = correct / total
accuracy

0.7498718605843158

# Few Shots (shots = 3)

English: 82.03%, used 424k tokens. run time 34min

Chinese: 75.04%, 567k tokens, 19min

In [33]:
few_shot_prompt_en = """Analyze the following text and determine if it describes a real event or is likely fabricated.
Output '0' if it is likely real.
Output '1' if it is likely fake.
Respond *only* with the digit 0 or 1.

Here are some examples:

Text: US immigration judge rules Palestinian Columbia student Khalil can be deported.
Output: 0

Text: A law enforcement sniper assigned to former President Donald Trump’s rally Saturday in Butler, Pennsylvania, says the head of the Secret Service ordered him not to shoot the suspect accused of attempting to assassinate Trump.
Output: 1

Text: The Dane County Farmers Market kicked off its 53rd season Saturday under a sunny sky, brisk but rising temperatures and no worries of tariffs.
Output: 0

Now, classify the following text:

Text:
{content}
Output:




"""


In [53]:
en_result = []
predictions = []

for item in dataset['en_test']:
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": few_shot_prompt_en},
                    {"role": "user", "content": item['content']},
                ],
                temperature=0.1,
                max_tokens=1,
            )
            break  # break the loop if no exception
        except openai.error.RateLimitError:
            print("Rate limit hit, sleeping for 1 second...")
            time.sleep(1)  # wait and retry

    prediction = response['choices'][0]['message']['content'].strip()
    predictions.append(prediction)

    label = item['label']

    en_result.append({
        "content": item['content'],
        "label": label,
        "prediction": prediction
    })

# Save to JSON
with open("fewshots_en_results.json", "w", encoding="utf-8") as f:
    json.dump({"en_result": en_result}, f, indent=2, ensure_ascii=False)


Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, 

In [63]:
with open("fewshots_en_results.json", "r", encoding="utf-8") as f:
    fewshots_en_results = json.load(f)

for item in fewshots_en_results['en_result']:
    item['match'] = str(item['label']) == item['prediction']



In [64]:

# Compute accuracy
total = len(results)
correct = sum(item['match'] for item in fewshots_en_results['en_result'])
accuracy = correct / total

accuracy

0.8203497615262321

In [65]:
few_shot_prompt_ch = """分析以下文本，判断它描述的是真实事件还是可能为虚构。
如果可能为真，输出 '0'。
如果可能为假，输出 '1'。
请*仅*回应数字 0 或 1。

这里有一些例子：

文本：开局良好，多组数据折射我国一季度经济运行向好态势。国家信息中心监测数据显示，2月挖掘机销售台数同比增长52.8%，其中国内同比增长99.4%，为4年来最大单月涨幅.
输出：0

文本：大阪世博会中国馆亮点抢先看：搭乘 “蛟龙”号，探寻深海奥秘。
输出：0

文本：台湾花莲县海域发生7.3级地震，导致浙江宁波一小区外墙开裂
输出：1

现在，请对以下文本进行分类：

文本：
{content}
输出：

"""


In [78]:
ch_result = []
predictions = []

for item in dataset['ch_test']:
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": few_shot_prompt_ch},
                    {"role": "user", "content": item['content']},
                ],
                temperature=0.1,
                max_tokens=1,
            )
            break  # break the loop if no exception
        except openai.error.RateLimitError:
            print("Rate limit hit, sleeping for 1 second...")
            time.sleep(1)  # wait and retry

    prediction = response['choices'][0]['message']['content'].strip()
    predictions.append(prediction)

    label = item['label']

    ch_result.append({
        "content": item['content'],
        "label": label,
        "prediction": prediction
    })

# Save to JSON
with open("fewshots_ch_results.json", "w", encoding="utf-8") as f:
    json.dump({"ch_result": ch_result}, f, indent=2, ensure_ascii=False)


Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, sleeping for 1 second...
Rate limit hit, 

In [79]:
with open("fewshots_ch_results.json", "r", encoding="utf-8") as f:
    fewshots_ch_results = json.load(f)

for item in fewshots_ch_results['ch_result']:
    item['match'] = str(item['label']) == item['prediction']
fewshots_ch_results

{'ch_result': [{'content': '全国首家无人银行亮相大堂经理是智能机器人银行里没有忙碌的工作人员和拥挤的排队客户，而是各种机器人、智慧柜员机、存取款机、外汇兑换机等智能化设备，科技感十足。业内观点无人银行是银行物理网点的一个创新，是转型的一种选择，但并不意味着会成为一种趋势，线上银行才是未来银行业发展开全文c',
   'label': 0,
   'prediction': '0',
   'match': True},
  {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
   'label': 0,
   'prediction': '0',
   'match': True},
  {'content': '【vivo隐形指纹技术正式亮相】全面屏手机时代即将到来。如此一来前后指纹已成为过去式，所有手机设计上又将站在同一起跑线了，大家觉得屏幕内指纹解锁帅不帅？秒拍视频',
   'label': 0,
   'prediction': '0',
   'match': True},
  {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
   'label': 0,
   'prediction': '0',
   'match': True},
  {'content': '【在5g、物联网、人工智能等领域开展合作，华为与湖南郴州签署战略合作协议】根据战略协议，双方将积极推动新基建项目建设，在新型智慧城市、5g、云计算、大数据、物联网、平安城市、人工智能、工业互联网等领域开展合作...在5g、物联网、人工智能等领域开展合作，华为与湖南郴州签署战略合作协议',
   'label': 0,
   'prediction': '0',
   'match': T

In [80]:


# Compute accuracy
total = len(fewshots_ch_results['ch_result'])
correct = sum(item['match'] for item in fewshots_ch_results['ch_result'])
accuracy = correct / total

accuracy

0.7503844182470528

In [148]:
with open("fewshots_en_results.json", "r", encoding="utf-8") as f:
    fewshots_en_results = json.load(f)['en_result']

# Filter entries with prediction == -1
filtered_data = [item for item in fewshots_en_results if item['prediction'] != -1]


total = len(filtered_data)
correct = sum(1 for item in ch_result if item['match'])
accuracy = correct / total
accuracy

0.5603305785123966

# Few Shots + Cot

English: 663 tokens, 72 min, 84.02% acc

Chinese:  1104 tokens, 78 min, 77.91% acc

In [81]:
few_shot_cot_prompt_en = """Analyze the following text and determine if it describes a real event or is likely fabricated.
First, provide a step-by-step reasoning for your decision.
Then, on a new line, provide the final answer in the format 'Final Answer: 0' if it is likely real, or 'Final Answer: 1' if it is likely fake.

Here are some examples demonstrating the process:

Text: US immigration judge rules Palestinian Columbia student Khalil can be deported.
Reasoning: The text reports a specific legal decision involving identifiable entities (US immigration judge, Columbia student Khalil) and a common legal action (deportation ruling). This aligns with typical news reporting formats for factual events.
Final Answer: 0

Text: A law enforcement sniper assigned to former President Donald Trump’s rally Saturday in Butler, Pennsylvania, says the head of the Secret Service ordered him not to shoot the suspect accused of attempting to assassinate Trump.
Reasoning: The text makes an extraordinary and highly sensitive claim (an order not to shoot an assassination suspect) attributed to the head of the Secret Service via a sniper. Such a claim is sensational, lacks immediate corroboration within the text, involves high-level security protocols that make this scenario unlikely, and borders on conspiracy theory.
Final Answer: 1

Text: The Dane County Farmers Market kicked off its 53rd season Saturday under a sunny sky, brisk but rising temperatures and no worries of tariffs.
Reasoning: The text describes a routine, verifiable local event (Dane County Farmers Market opening). It includes specific details (53rd season, location, weather) characteristic of factual local news reporting.
Final Answer: 0

Now, analyze the following text using the same process:

Text:
{content}

Reasoning:
[Provide your reasoning steps here]

Final Answer:
[0 or 1]



"""


In [99]:
en_result = []
predictions = []

for item in dataset['en_test']:
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": few_shot_cot_prompt_en},
                    {"role": "user", "content": item['content']},
                ],
                temperature=0.1,
            )
            break  # break the loop if no exception
        except openai.error.RateLimitError:
            print("Rate limit hit, sleeping for 1 second...")
            time.sleep(1)  # wait and retry

    prediction = response['choices'][0]['message']['content'].strip()
    predictions.append(prediction)

    label = item['label']

    en_result.append({
        "content": item['content'],
        "label": label,
        "result": prediction
    })

# Save to JSON
with open("fewshots_cot_en_results.json", "w", encoding="utf-8") as f:
    json.dump({"en_result": en_result}, f, indent=2, ensure_ascii=False)


In [100]:
with open("fewshots_cot_en_results.json", "r", encoding="utf-8") as f:
    fewshots_cot_en_results = json.load(f)



# extract the result

for item in fewshots_cot_en_results['en_result']:
    # Extract the final digit after 'Final Answer:'
    result_text = item['result']
    if 'Final Answer:' in result_text:
        item['prediction'] = result_text.strip().split('Final Answer:')[-1].strip()[-1]
    else:
        item['prediction'] = None  # Fallback in case the format is unexpected


# match or not 
for item in fewshots_cot_en_results['en_result']:
    item['match'] = str(item['label']) == item['prediction']



# Compute accuracy
total = len(fewshots_cot_en_results['en_result'])
correct = sum(item['match'] for item in fewshots_cot_en_results['en_result'])
accuracy = correct / total

accuracy

0.8402225755166932

In [102]:
few_shot_cot_prompt_ch ="""

分析以下文本，判断它描述的是真实事件还是可能为虚构。
首先，请提供你做此判断的分步推理过程。
然后，在新的一行，按照以下格式提供最终答案：如果可能为真，则为 'Final Answer: 0'；如果可能为假，则为 'Final Answer: 1'。

这里有一些演示该过程的例子：

文本：开局良好，多组数据折射我国一季度经济运行向好态势。国家信息中心监测数据显示，2月挖掘机销售台数同比增长52.8%，其中国内同比增长99.4%，为4年来最大单月涨幅.
推理过程：该文本提供了具体的经济数据（挖掘机销售数量、增长率），引用了数据来源（国家信息中心），并进行了比较（4年来最大单月涨幅）。这符合标准经济新闻报道的格式，内容具有可验证性。
Final Answer: 0

文本：大阪世博会中国馆亮点抢先看：搭乘 “蛟龙”号，探寻深海奥秘。
推理过程：该文本描述的是对未来活动（大阪世博会中国馆）的一个特色预览。虽然涉及具体技术（“蛟龙”号），但本质是宣传性内容，介绍的是一个计划中的、真实的活动亮点。
Final Answer: 0

文本：台湾花莲县海域发生7.3级地震，导致浙江宁波一小区外墙开裂
推理过程：文本声称一个地点（台湾花莲）的地震导致了非常遥远的另一个地点（浙江宁波）发生具体的物理损坏（外墙开裂）。虽然强震影响广泛，但这种跨越巨大地理距离的直接、具体的因果关系（特定小区外墙因此开裂）高度可疑，缺乏合理的物理传导机制解释。
Final Answer: 1

现在，请使用相同的过程分析以下文本：

文本：
{content}

推理过程：
[在此处提供你的推理步骤]

Final Answer:
[0 或 1]
"""

In [105]:
ch_result = []
predictions = []

for item in dataset['ch_test']:
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": few_shot_cot_prompt_ch},
                    {"role": "user", "content": item['content']},
                ],
                temperature=0.1,
            )
            break  # break the loop if no exception
        except openai.error.RateLimitError:
            print("Rate limit hit, sleeping for 1 second...")
            time.sleep(1)  # wait and retry

    prediction = response['choices'][0]['message']['content'].strip()
    predictions.append(prediction)

    label = item['label']

    ch_result.append({
        "content": item['content'],
        "label": label,
        "result": prediction
    })

# Save to JSON
with open("fewshots_cot_ch_results.json", "w", encoding="utf-8") as f:
    json.dump({"ch_result": ch_result}, f, indent=2, ensure_ascii=False)


In [106]:
with open("fewshots_cot_ch_results.json", "r", encoding="utf-8") as f:
    fewshots_cot_ch_results = json.load(f)



# extract the result

for item in fewshots_cot_ch_results['ch_result']:
    # Extract the final digit after 'Final Answer:'
    result_text = item['result']
    if 'Final Answer:' in result_text:
        item['prediction'] = result_text.strip().split('Final Answer:')[-1].strip()[-1]
    else:
        item['prediction'] = None  # Fallback in case the format is unexpected


# match or not 
for item in fewshots_cot_ch_results['ch_result']:
    item['match'] = str(item['label']) == item['prediction']



# Compute accuracy
total = len(fewshots_cot_ch_results['ch_result'])
correct = sum(item['match'] for item in fewshots_cot_ch_results['ch_result'])
accuracy = correct / total

accuracy

0.7780625320348539